# Experiment with recognizing times from MK8DX screenshots

In [6]:
import tempfile
from PIL import Image
from pathlib import Path
import os
import sys
import io
import datetime as dt
import subprocess

## Identify which kind of image we have

We need a way to distinguish among at least three image formats:
1. Blue background, track name at bottom, combo listed with text.
2. Track background, racing alone (no ghost).
3. Track background, racing against ghost.

We also need a way to identify how many laps there are in the race. This can be either three or seven, with three being the most common.

### Determine kind of image

In [17]:
class ImageToOCR():
    def __init__(self, image_file_name):
        self.image_file_name = image_file_name
    
    @property
    def ocr(self) -> str:
        ocr_result = subprocess.run(['ocrs', self.image_file_name], stdout=subprocess.PIPE)
        return ocr_result.stdout.decode('utf-8').strip()
    
    def ocr_box(self, box: list[4]) -> str:
        """
        Given a box defining opposite corners of a box in pixels,
        return the text ocr'ed from that box.

        box = a list of integers: [x1, y1, x2, y2]
        """
        original_image = Image.open(self.image_file_name)
        cropped_image = original_image.crop(box)
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temporary_file:
            temporary_file_name = temporary_file.name
        cropped_image.save(temporary_file_name)
        ocr_result = subprocess.run(['ocrs', temporary_file_name], stdout=subprocess.PIPE)
        
        return ocr_result.stdout.decode('utf-8').strip()
    
    @property
    def kind(self) -> int:
        with Image.open(self.image_file_name) as f:
            image_size = f.size
        
        lower_left_button = self.ocr_box(box=[109, 647, 138, 676])
        lower_right_lap_2_number = self.ocr_box(box=[1020, 456, 1044, 480])
        last_digit_top_lap_2 = self.ocr_box(box=[1162, 220, 1183, 253])

        if image_size != (1280, 720):
            # Wrong size; can't be a screen capture from the Switch
            image_kind = 0
        elif lower_left_button == 'B':
            # Looks for the B button in the lower left corner
            image_kind = 1
        elif lower_right_lap_2_number == '2':
            # Looks for the second lap in the lower box of times.
            # Assumes that the course has three laps.
            image_kind = 3
        elif len(last_digit_top_lap_2) is not None:
            # If there's no lower box of times, looks for the second lap
            # in the upper box of times.
            image_kind = 2
        else:
            # Can't positively identify as one of the defined kinds.
            image_kind = 0
        return image_kind
    
    def pull_data_from_image(self, kind=None) -> dict:
        """
        Pulls information from image. Returns a dictionary of strings.
        """
        if kind == None:
            return self.pull_data_from_image(kind=self.kind)
        if kind not in [1,2,3]:
            return {}
        if kind == 1:
            racer = self.ocr_box(box=[788, 110, 1074, 143])
            overall_time = self.ocr_box(box=[870, 144, 1074, 182])
            lap_times = {
                1: self.ocr_box(box=[866, 210, 1026, 250]),
                2: self.ocr_box(box=[866, 250, 1026, 290]),
                3: self.ocr_box(box=[866, 290, 1026, 330])
            }
            vehicle = self.ocr_box(box=[830, 350, 1100, 390])
            wheels = self.ocr_box(box=[830, 410, 1100, 450])
            glider = self.ocr_box(box=[830, 470, 1100, 510])
            track = self.ocr_box(box=[300, 640, 830, 690])
            output = {
                'racer': racer,
                'overall_time': overall_time,
                'lap_times': lap_times,
                'vehicle': vehicle,
                'wheels': wheels,
                'glider': glider,
                'track': track
            }
            return output

In [18]:
# kind 1
image_file = 'data/test_images/2019102710534900_c.jpg'
print(image_file + ': ' + str(ImageToOCR(image_file).kind))
print(ImageToOCR(image_file).pull_data_from_image(1))

# kind 2
image_file = 'data/test_images/2023062418352400_s.jpg'
print(image_file + ': ' + str(ImageToOCR(image_file).kind))
print(ImageToOCR(image_file).pull_data_from_image(2))

# kind 3
image_file = 'data/test_images/2023070714422000_s.jpg'
print(image_file + ': ' + str(ImageToOCR(image_file).kind))
print(ImageToOCR(image_file).pull_data_from_image(3))

# kind other
image_file = 'data/test_images/dog.jpg'
print(image_file + ': ' + str(ImageToOCR(image_file).kind))
print(ImageToOCR(image_file).pull_data_from_image(0))


data/test_images/2019102710534900_c.jpg: 2
{'racer': 'Ander', 'overall_time': '124739', 'lap_times': {1: '0:28.73', 2: '0:28.33', 3: '0:27.7'}, 'vehicle': 'Biddybuggy', 'wheels': 'Roller', 'glider': 'Gold Glider', 'track': 'GCN Yoshi Circuit'}
data/test_images/2023062418352400_s.jpg: 2
None
data/test_images/2023070714422000_s.jpg: 3
None
data/test_images/dog.jpg: 0
{}
